# Deep Learning For Rossmann Competition

## Goals

https://www.kaggle.com/c/rossmann-store-sales

Rossmann store managers are tasked with predicting their daily sales for up to six weeks in advance.
Rossmann is challenging you to predict 6 weeks of daily sales for 1,115 stores located across Germany.

Reliable sales forecasts enable store managers to create effective staff schedules that increase productivity and motivation.

## Evaluation

Submissions are evaluated on the Root Mean Square Percentage Error (RMSPE).

## Initial imports

In [4]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [5]:
from fastai.structured import *
from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=20)

PATH='data/rossmann-store-sales-additional/'

## Dataset

The dataset can be found here: http://files.fast.ai/part2/lesson14/rossmann.tgz

- train: training set provided by competition
- store: list of stores
- store_states: mapping of store to the German state they are in
- state_names: list of German state names
- googletrend: trend of certain google keywords over time
- weather: weather info
- test: testing set

In [6]:
table_names = ['train', 'store', 'store_states', 'state_names', 'googletrend', 'weather', 'test']

Load all of the csv files as dataframes into the list `tables`

In [7]:
tables = [pd.read_csv(f'{PATH}{fname}.csv', low_memory=False) for fname in table_names]

Displaying first couple of rows of each table

In [8]:
from IPython.display import HTML, display

In [9]:
for t in tables: display(t.head())

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


,Store,State
0,1,HE
1,2,TH
2,3,NW
3,4,BE
4,5,SN


,StateName,State
0,BadenWuerttemberg,BW
1,Bayern,BY
2,Berlin,BE
3,Brandenburg,BB
4,Bremen,HB


,file,week,trend
0,Rossmann_DE_SN,2012-12-02 - 2012-12-08,96
1,Rossmann_DE_SN,2012-12-09 - 2012-12-15,95
2,Rossmann_DE_SN,2012-12-16 - 2012-12-22,91
3,Rossmann_DE_SN,2012-12-23 - 2012-12-29,48
4,Rossmann_DE_SN,2012-12-30 - 2013-01-05,67


,file,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,...,Max_VisibilityKm,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees
0,NordrheinWestfalen,2013-01-01,8,4,2,7,5,1,94,87,...,31.0,12.0,4.0,39,26,58.0,5.08,6.0,Rain,215
1,NordrheinWestfalen,2013-01-02,7,4,1,5,3,2,93,85,...,31.0,14.0,10.0,24,16,NaN,0.00,6.0,Rain,225
2,NordrheinWestfalen,2013-01-03,11,8,6,10,8,4,100,93,...,31.0,8.0,2.0,26,21,NaN,1.02,7.0,Rain,240
3,NordrheinWestfalen,2013-01-04,9,9,8,9,9,8,100,94,...,11.0,5.0,2.0,23,14,NaN,0.25,7.0,Rain,263
4,NordrheinWestfalen,2013-01-05,8,8,7,8,7,6,100,94,...,10.0,6.0,3.0,16,10,NaN,0.00,7.0,Rain,268


,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


Displaying summarized aggregate information on each table

In [10]:
# this line has a problem with newer version of pandas
#for t in tables: display(DataFrameSummary(t).summary())

## Data Cleaning and Feature Engineering

In [11]:
train, store, store_states, state_names, googletrend, weather, test = tables

In [12]:
len(train),len(test)

(1017209, 41088)

Converting state holidays to boolean:

In [13]:
# please note - these lines throw a warning if they are executed twice
train.StateHoliday = ( train.StateHoliday != '0' )
test.StateHoliday = ( test.StateHoliday != '0')

Implementing a new function for joining tables on specific fields.

In [14]:
def join_df(left, right, left_join_field_name, right_join_field_name=None, right_duplicate_field_suffix='_y'):
    if right_join_field_name is None: right_join_field_name = left_join_field_name
    return left.merge(
        right,
        how='left',
        left_on=left_join_field_name,
        right_on=right_join_field_name,
        suffixes=('', right_duplicate_field_suffix)
    )

Join weather with state names

In [15]:
weather

,file,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,...,Max_VisibilityKm,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees
0,NordrheinWestfalen,2013-01-01,8,4,2,7,5,1,94,87,...,31.0,12.0,4.0,39,26,58.0,5.08,6.0,Rain,215
1,NordrheinWestfalen,2013-01-02,7,4,1,5,3,2,93,85,...,31.0,14.0,10.0,24,16,NaN,0.00,6.0,Rain,225
2,NordrheinWestfalen,2013-01-03,11,8,6,10,8,4,100,93,...,31.0,8.0,2.0,26,21,NaN,1.02,7.0,Rain,240
3,NordrheinWestfalen,2013-01-04,9,9,8,9,9,8,100,94,...,11.0,5.0,2.0,23,14,NaN,0.25,7.0,Rain,263
4,NordrheinWestfalen,2013-01-05,8,8,7,8,7,6,100,94,...,10.0,6.0,3.0,16,10,NaN,0.00,7.0,Rain,268
5,NordrheinWestfalen,2013-01-06,9,8,7,8,7,6,100,92,...,18.0,7.0,5.0,16,11,NaN,0.00,7.0,Rain,258
6,NordrheinWestfalen,2013-01-07,9,8,7,8,7,5,98,88,...,26.0,8.0,2.0,16,10,NaN,0.00,7.0,Rain,238
7,NordrheinWestfalen,2013-01-08,8,7,5,6,5,4,100,87,...,18.0,9.0,3.0,21,13,NaN,0.00,7.0,Rain,230
8,NordrheinWestfalen,2013-01-09,6,6,5,6,4,3,100,91,...,19.0,8.0,2.0,21,14,NaN,4.06,8.0,Rain,222
9,NordrheinWestfalen,2013-01-10,6,4,3,4,3,1,96,86,...,26.0,12.0,6.0,16,11,NaN,0.00,6.0,Rain,256


In [16]:
state_names

,StateName,State
0,BadenWuerttemberg,BW
1,Bayern,BY
2,Berlin,BE
3,Brandenburg,BB
4,Bremen,HB
5,Hamburg,HH
6,Hessen,HE
7,MecklenburgVorpommern,MV
8,Niedersachsen,"HB,NI"
9,NordrheinWestfalen,NW


In [17]:
weather = join_df(weather, state_names, 'file', 'StateName')

In [18]:
weather

,file,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,...,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees,StateName,State
0,NordrheinWestfalen,2013-01-01,8,4,2,7,5,1,94,87,...,4.0,39,26,58.0,5.08,6.0,Rain,215,NordrheinWestfalen,NW
1,NordrheinWestfalen,2013-01-02,7,4,1,5,3,2,93,85,...,10.0,24,16,NaN,0.00,6.0,Rain,225,NordrheinWestfalen,NW
2,NordrheinWestfalen,2013-01-03,11,8,6,10,8,4,100,93,...,2.0,26,21,NaN,1.02,7.0,Rain,240,NordrheinWestfalen,NW
3,NordrheinWestfalen,2013-01-04,9,9,8,9,9,8,100,94,...,2.0,23,14,NaN,0.25,7.0,Rain,263,NordrheinWestfalen,NW
4,NordrheinWestfalen,2013-01-05,8,8,7,8,7,6,100,94,...,3.0,16,10,NaN,0.00,7.0,Rain,268,NordrheinWestfalen,NW
5,NordrheinWestfalen,2013-01-06,9,8,7,8,7,6,100,92,...,5.0,16,11,NaN,0.00,7.0,Rain,258,NordrheinWestfalen,NW
6,NordrheinWestfalen,2013-01-07,9,8,7,8,7,5,98,88,...,2.0,16,10,NaN,0.00,7.0,Rain,238,NordrheinWestfalen,NW
7,NordrheinWestfalen,2013-01-08,8,7,5,6,5,4,100,87,...,3.0,21,13,NaN,0.00,7.0,Rain,230,NordrheinWestfalen,NW
8,NordrheinWestfalen,2013-01-09,6,6,5,6,4,3,100,91,...,2.0,21,14,NaN,4.06,8.0,Rain,222,NordrheinWestfalen,NW
9,NordrheinWestfalen,2013-01-10,6,4,3,4,3,1,96,86,...,6.0,16,11,NaN,0.00,6.0,Rain,256,NordrheinWestfalen,NW


Adding date and state field to the googletrend dataframe.

In [19]:
googletrend

,file,week,trend
0,Rossmann_DE_SN,2012-12-02 - 2012-12-08,96
1,Rossmann_DE_SN,2012-12-09 - 2012-12-15,95
2,Rossmann_DE_SN,2012-12-16 - 2012-12-22,91
3,Rossmann_DE_SN,2012-12-23 - 2012-12-29,48
4,Rossmann_DE_SN,2012-12-30 - 2013-01-05,67
5,Rossmann_DE_SN,2013-01-06 - 2013-01-12,71
6,Rossmann_DE_SN,2013-01-13 - 2013-01-19,53
7,Rossmann_DE_SN,2013-01-20 - 2013-01-26,58
8,Rossmann_DE_SN,2013-01-27 - 2013-02-02,55
9,Rossmann_DE_SN,2013-02-03 - 2013-02-09,55


In pandas you can add new columns to a dataframe by simply defining it. We'll do this for googletrends by extracting dates and state names from the given data and adding those columns.

We're also going to replace all instances of state name 'NI' to match the usage in the rest of the data: 'HB,NI'. This is a good opportunity to highlight pandas indexing. We can use .loc[rows, cols] to select a list of rows and a list of columns from the dataframe. In this case, we're selecting rows w/ statename 'NI' by using a boolean list googletrend.State=='NI' and selecting "State".

In [20]:
googletrend['Date'] = googletrend.week.str.split(' - ', expand=True)[0]
googletrend['State'] = googletrend.file.str.split('_', expand=True)[2]
googletrend.loc[googletrend.State=='NI', "State"] = 'HB,NI'

In [21]:
googletrend

,file,week,trend,Date,State
0,Rossmann_DE_SN,2012-12-02 - 2012-12-08,96,2012-12-02,SN
1,Rossmann_DE_SN,2012-12-09 - 2012-12-15,95,2012-12-09,SN
2,Rossmann_DE_SN,2012-12-16 - 2012-12-22,91,2012-12-16,SN
3,Rossmann_DE_SN,2012-12-23 - 2012-12-29,48,2012-12-23,SN
4,Rossmann_DE_SN,2012-12-30 - 2013-01-05,67,2012-12-30,SN
5,Rossmann_DE_SN,2013-01-06 - 2013-01-12,71,2013-01-06,SN
6,Rossmann_DE_SN,2013-01-13 - 2013-01-19,53,2013-01-13,SN
7,Rossmann_DE_SN,2013-01-20 - 2013-01-26,58,2013-01-20,SN
8,Rossmann_DE_SN,2013-01-27 - 2013-02-02,55,2013-01-27,SN
9,Rossmann_DE_SN,2013-02-03 - 2013-02-09,55,2013-02-03,SN


Extracting particular date fields from a complete datetime for a purpose of constructing categories.

In [22]:
weather

,file,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,...,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees,StateName,State
0,NordrheinWestfalen,2013-01-01,8,4,2,7,5,1,94,87,...,4.0,39,26,58.0,5.08,6.0,Rain,215,NordrheinWestfalen,NW
1,NordrheinWestfalen,2013-01-02,7,4,1,5,3,2,93,85,...,10.0,24,16,NaN,0.00,6.0,Rain,225,NordrheinWestfalen,NW
2,NordrheinWestfalen,2013-01-03,11,8,6,10,8,4,100,93,...,2.0,26,21,NaN,1.02,7.0,Rain,240,NordrheinWestfalen,NW
3,NordrheinWestfalen,2013-01-04,9,9,8,9,9,8,100,94,...,2.0,23,14,NaN,0.25,7.0,Rain,263,NordrheinWestfalen,NW
4,NordrheinWestfalen,2013-01-05,8,8,7,8,7,6,100,94,...,3.0,16,10,NaN,0.00,7.0,Rain,268,NordrheinWestfalen,NW
5,NordrheinWestfalen,2013-01-06,9,8,7,8,7,6,100,92,...,5.0,16,11,NaN,0.00,7.0,Rain,258,NordrheinWestfalen,NW
6,NordrheinWestfalen,2013-01-07,9,8,7,8,7,5,98,88,...,2.0,16,10,NaN,0.00,7.0,Rain,238,NordrheinWestfalen,NW
7,NordrheinWestfalen,2013-01-08,8,7,5,6,5,4,100,87,...,3.0,21,13,NaN,0.00,7.0,Rain,230,NordrheinWestfalen,NW
8,NordrheinWestfalen,2013-01-09,6,6,5,6,4,3,100,91,...,2.0,21,14,NaN,4.06,8.0,Rain,222,NordrheinWestfalen,NW
9,NordrheinWestfalen,2013-01-10,6,4,3,4,3,1,96,86,...,6.0,16,11,NaN,0.00,6.0,Rain,256,NordrheinWestfalen,NW


In [23]:
add_datepart(weather, "Date", drop=False)

In [24]:
weather

,file,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,...,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,NordrheinWestfalen,2013-01-01,8,4,2,7,5,1,94,87,...,1,1,1,False,True,False,True,False,True,1356998400
1,NordrheinWestfalen,2013-01-02,7,4,1,5,3,2,93,85,...,2,2,2,False,False,False,False,False,False,1357084800
2,NordrheinWestfalen,2013-01-03,11,8,6,10,8,4,100,93,...,3,3,3,False,False,False,False,False,False,1357171200
3,NordrheinWestfalen,2013-01-04,9,9,8,9,9,8,100,94,...,4,4,4,False,False,False,False,False,False,1357257600
4,NordrheinWestfalen,2013-01-05,8,8,7,8,7,6,100,94,...,5,5,5,False,False,False,False,False,False,1357344000
5,NordrheinWestfalen,2013-01-06,9,8,7,8,7,6,100,92,...,6,6,6,False,False,False,False,False,False,1357430400
6,NordrheinWestfalen,2013-01-07,9,8,7,8,7,5,98,88,...,7,0,7,False,False,False,False,False,False,1357516800
7,NordrheinWestfalen,2013-01-08,8,7,5,6,5,4,100,87,...,8,1,8,False,False,False,False,False,False,1357603200
8,NordrheinWestfalen,2013-01-09,6,6,5,6,4,3,100,91,...,9,2,9,False,False,False,False,False,False,1357689600
9,NordrheinWestfalen,2013-01-10,6,4,3,4,3,1,96,86,...,10,3,10,False,False,False,False,False,False,1357776000


And the same for other dataframes as well:

In [25]:
add_datepart(googletrend, "Date", drop=False)
add_datepart(train, "Date", drop=False)
add_datepart(test, "Date", drop=False)

The Google trends data has a special category for the whole of the Germany - we'll pull that out so we can use it explicitly.

In [26]:
googletrend

,file,week,trend,Date,State,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,Rossmann_DE_SN,2012-12-02 - 2012-12-08,96,2012-12-02,SN,2012,12,48,2,6,337,False,False,False,False,False,False,1354406400
1,Rossmann_DE_SN,2012-12-09 - 2012-12-15,95,2012-12-09,SN,2012,12,49,9,6,344,False,False,False,False,False,False,1355011200
2,Rossmann_DE_SN,2012-12-16 - 2012-12-22,91,2012-12-16,SN,2012,12,50,16,6,351,False,False,False,False,False,False,1355616000
3,Rossmann_DE_SN,2012-12-23 - 2012-12-29,48,2012-12-23,SN,2012,12,51,23,6,358,False,False,False,False,False,False,1356220800
4,Rossmann_DE_SN,2012-12-30 - 2013-01-05,67,2012-12-30,SN,2012,12,52,30,6,365,False,False,False,False,False,False,1356825600
5,Rossmann_DE_SN,2013-01-06 - 2013-01-12,71,2013-01-06,SN,2013,1,1,6,6,6,False,False,False,False,False,False,1357430400
6,Rossmann_DE_SN,2013-01-13 - 2013-01-19,53,2013-01-13,SN,2013,1,2,13,6,13,False,False,False,False,False,False,1358035200
7,Rossmann_DE_SN,2013-01-20 - 2013-01-26,58,2013-01-20,SN,2013,1,3,20,6,20,False,False,False,False,False,False,1358640000
8,Rossmann_DE_SN,2013-01-27 - 2013-02-02,55,2013-01-27,SN,2013,1,4,27,6,27,False,False,False,False,False,False,1359244800
9,Rossmann_DE_SN,2013-02-03 - 2013-02-09,55,2013-02-03,SN,2013,2,5,3,6,34,False,False,False,False,False,False,1359849600


In [27]:
trend_de = googletrend[googletrend.file == 'Rossmann_DE']

In [28]:
trend_de

,file,week,trend,Date,State,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
148,Rossmann_DE,2012-12-02 - 2012-12-08,77,2012-12-02,None,2012,12,48,2,6,337,False,False,False,False,False,False,1354406400
149,Rossmann_DE,2012-12-09 - 2012-12-15,76,2012-12-09,None,2012,12,49,9,6,344,False,False,False,False,False,False,1355011200
150,Rossmann_DE,2012-12-16 - 2012-12-22,85,2012-12-16,None,2012,12,50,16,6,351,False,False,False,False,False,False,1355616000
151,Rossmann_DE,2012-12-23 - 2012-12-29,59,2012-12-23,None,2012,12,51,23,6,358,False,False,False,False,False,False,1356220800
152,Rossmann_DE,2012-12-30 - 2013-01-05,61,2012-12-30,None,2012,12,52,30,6,365,False,False,False,False,False,False,1356825600
153,Rossmann_DE,2013-01-06 - 2013-01-12,62,2013-01-06,None,2013,1,1,6,6,6,False,False,False,False,False,False,1357430400
154,Rossmann_DE,2013-01-13 - 2013-01-19,51,2013-01-13,None,2013,1,2,13,6,13,False,False,False,False,False,False,1358035200
155,Rossmann_DE,2013-01-20 - 2013-01-26,56,2013-01-20,None,2013,1,3,20,6,20,False,False,False,False,False,False,1358640000
156,Rossmann_DE,2013-01-27 - 2013-02-02,50,2013-01-27,None,2013,1,4,27,6,27,False,False,False,False,False,False,1359244800
157,Rossmann_DE,2013-02-03 - 2013-02-09,58,2013-02-03,None,2013,2,5,3,6,34,False,False,False,False,False,False,1359849600


Now we can outer join all of our data into a single dataframe. Recall that in outer joins everytime a value in the joining field on the left table does not have a corresponding value on the right table, the corresponding row in the new table has Null values for all right table fields. One way to check that all records are consistent and complete is to check for Null values post-join, as we do here.

Aside: Why not just do an inner join? If you are assuming that all records are complete and match on the field you desire, an inner join will do the same thing as an outer join. However, in the event you are wrong or a mistake is made, an outer join followed by a null-check will catch it. (Comparing before/after # of rows for inner join is equivalent, but requires keeping track of before/after row #'s. Outer join is easier.)

In [29]:
store

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN
5,6,a,a,310.0,12.0,2013.0,0,NaN,NaN,NaN
6,7,a,c,24000.0,4.0,2013.0,0,NaN,NaN,NaN
7,8,a,a,7520.0,10.0,2014.0,0,NaN,NaN,NaN
8,9,a,c,2030.0,8.0,2000.0,0,NaN,NaN,NaN
9,10,a,a,3160.0,9.0,2009.0,0,NaN,NaN,NaN


In [30]:
store_states

,Store,State
0,1,HE
1,2,TH
2,3,NW
3,4,BE
4,5,SN
5,6,SN
6,7,SH
7,8,SH
8,9,NW
9,10,"HB,NI"


In [31]:
store = join_df(store, store_states, "Store")
len(store[store.State.isnull()])

0

In [32]:
store

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,State
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,HE
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",TH
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",NW
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN,BE
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN,SN
5,6,a,a,310.0,12.0,2013.0,0,NaN,NaN,NaN,SN
6,7,a,c,24000.0,4.0,2013.0,0,NaN,NaN,NaN,SH
7,8,a,a,7520.0,10.0,2014.0,0,NaN,NaN,NaN,SH
8,9,a,c,2030.0,8.0,2000.0,0,NaN,NaN,NaN,NW
9,10,a,a,3160.0,9.0,2009.0,0,NaN,NaN,NaN,"HB,NI"


In [33]:
joined = join_df(train, store, "Store")
joined_test = join_df(test, store, "Store")
len(joined[joined.StoreType.isnull()]),len(joined_test[joined_test.StoreType.isnull()])

(0, 0)

In [34]:
joined = join_df(joined, googletrend, ["State","Year", "Week"])
joined_test = join_df(joined_test, googletrend, ["State","Year", "Week"])
len(joined[joined.trend.isnull()]),len(joined_test[joined_test.trend.isnull()])

(0, 0)

In [35]:
joined = joined.merge(trend_de, 'left', ["Year", "Week"], suffixes=('', '_DE'))
joined_test = joined_test.merge(trend_de, 'left', ["Year", "Week"], suffixes=('', '_DE'))
len(joined[joined.trend_DE.isnull()]),len(joined_test[joined_test.trend_DE.isnull()])

(0, 0)

In [36]:
joined = join_df(joined, weather, ["State","Date"])
joined_test = join_df(joined_test, weather, ["State","Date"])
len(joined[joined.Mean_TemperatureC.isnull()]),len(joined_test[joined_test.Mean_TemperatureC.isnull()])

(0, 0)

In [37]:
for df in (joined, joined_test):
    for c in df.columns:
        if c.endswith('_y'):
            if c in df.columns: df.drop(c, inplace=True, axis=1)

Next we'll fill in missing values to avoid complications with NA's. NA (not available) is how Pandas indicates missing values; many models have problems when missing values are present, so it's always important to think about how to deal with them. In these cases, we are picking an arbitrary signal value that doesn't otherwise appear in the data.

In [38]:
for df in (joined,joined_test):
    df['CompetitionOpenSinceYear'] = df.CompetitionOpenSinceYear.fillna(1900).astype(np.int32)
    df['CompetitionOpenSinceMonth'] = df.CompetitionOpenSinceMonth.fillna(1).astype(np.int32)
    df['Promo2SinceYear'] = df.Promo2SinceYear.fillna(1900).astype(np.int32)
    df['Promo2SinceWeek'] = df.Promo2SinceWeek.fillna(1).astype(np.int32)

Next we'll extract features "CompetitionOpenSince" and "CompetitionDaysOpen". Note the use of apply() in mapping a function across dataframe values.

In [39]:
for df in (joined,joined_test):
    df["CompetitionOpenSince"] = pd.to_datetime(dict(year=df.CompetitionOpenSinceYear, 
                                                     month=df.CompetitionOpenSinceMonth, day=15))
    df["CompetitionDaysOpen"] = df.Date.subtract(df.CompetitionOpenSince).dt.days

We'll replace some erroneous / outlying data.

In [40]:
for df in (joined,joined_test):
    df.loc[df.CompetitionDaysOpen<0, "CompetitionDaysOpen"] = 0
    df.loc[df.CompetitionOpenSinceYear<1990, "CompetitionDaysOpen"] = 0

We add "CompetitionMonthsOpen" field, limiting the maximum to 2 years to limit number of unique categories.

In [41]:
for df in (joined,joined_test):
    df["CompetitionMonthsOpen"] = df["CompetitionDaysOpen"]//30
    df.loc[df.CompetitionMonthsOpen>24, "CompetitionMonthsOpen"] = 24
joined.CompetitionMonthsOpen.unique()

array([24,  3, 19,  9,  0, 16, 17,  7, 15, 22, 11, 13,  2, 23, 12,  4, 10,  1, 14, 20,  8, 18,  6, 21,  5])

Same process for Promo dates.

In [42]:
for df in (joined,joined_test):
    df["Promo2Since"] = pd.to_datetime(df.apply(lambda x: Week(
        x.Promo2SinceYear, x.Promo2SinceWeek).monday(), axis=1).astype(pd.datetime))
    df["Promo2Days"] = df.Date.subtract(df["Promo2Since"]).dt.days

In [43]:
for df in (joined,joined_test):
    df.loc[df.Promo2Days<0, "Promo2Days"] = 0
    df.loc[df.Promo2SinceYear<1990, "Promo2Days"] = 0
    df["Promo2Weeks"] = df["Promo2Days"]//7
    df.loc[df.Promo2Weeks<0, "Promo2Weeks"] = 0
    df.loc[df.Promo2Weeks>25, "Promo2Weeks"] = 25
    df.Promo2Weeks.unique()

Save data as feather format

In [44]:
joined.to_feather(f'{PATH}joined')
joined_test.to_feather(f'{PATH}joined_test')

## Durations

It is common when working with time series data to extract data that explains relationships across rows as opposed to columns, e.g.:

- Running averages
- Time until next event
- Time since last event

This is often difficult to do with most table manipulation frameworks, since they are designed to work with relationships across columns. As such, we've created a class to handle this type of data.

We'll define a function get_elapsed for cumulative counting across a sorted dataframe. Given a particular field fld to monitor, this function will start tracking time since the last occurrence of that field. When the field is seen again, the counter is set to zero.

Upon initialization, this will result in datetime na's until the field is encountered. This is reset every time a new store is seen. We'll see how to use this shortly.

In [45]:
def get_elapsed(fld, pre):
    day1 = np.timedelta64(1, 'D')
    last_date = np.datetime64()
    last_store = 0
    res = []

    for s,v,d in zip(df.Store.values,df[fld].values, df.Date.values):
        if s != last_store:
            last_date = np.datetime64()
            last_store = s
        if v: last_date = d
        res.append(((d-last_date).astype('timedelta64[D]') / day1))
    df[pre+fld] = res

We'll be applying this to a subset of columns:

In [46]:
columns = ["Date", "Store", "Promo", "StateHoliday", "SchoolHoliday"]

In [47]:
#df = train[columns]
df = train[columns].append(test[columns])

In [48]:
df

,Date,Store,Promo,StateHoliday,SchoolHoliday
0,2015-07-31,1,1,False,1
1,2015-07-31,2,1,False,1
2,2015-07-31,3,1,False,1
3,2015-07-31,4,1,False,1
4,2015-07-31,5,1,False,1
5,2015-07-31,6,1,False,1
6,2015-07-31,7,1,False,1
7,2015-07-31,8,1,False,1
8,2015-07-31,9,1,False,1
9,2015-07-31,10,1,False,1


Let's walk through an example.

Say we're looking at School Holiday. We'll first sort by Store, then Date, and then call add_elapsed('SchoolHoliday', 'After'): This will apply to each row with School Holiday:

A applied to every row of the dataframe in order of store and date
Will add to the dataframe the days since seeing a School Holiday
If we sort in the other direction, this will count the days until another holiday.

In [49]:
fld = 'SchoolHoliday'
df = df.sort_values(['Store', 'Date'])
get_elapsed(fld, 'After')
df = df.sort_values(['Store', 'Date'], ascending=[True, False])
get_elapsed(fld, 'Before')

In [50]:
df

,Date,Store,Promo,StateHoliday,SchoolHoliday,AfterSchoolHoliday,BeforeSchoolHoliday
0,2015-09-17,1,1,False,0,13.0,NaN
856,2015-09-16,1,1,False,0,12.0,NaN
1712,2015-09-15,1,1,False,0,11.0,NaN
2568,2015-09-14,1,1,False,0,10.0,NaN
3424,2015-09-13,1,0,False,0,9.0,NaN
4280,2015-09-12,1,0,False,0,8.0,NaN
5136,2015-09-11,1,0,False,0,7.0,NaN
5992,2015-09-10,1,0,False,0,6.0,NaN
6848,2015-09-09,1,0,False,0,5.0,NaN
7704,2015-09-08,1,0,False,0,4.0,NaN


We'll do this for two more fields.

In [51]:
fld = 'StateHoliday'
df = df.sort_values(['Store', 'Date'])
get_elapsed(fld, 'After')
df = df.sort_values(['Store', 'Date'], ascending=[True, False])
get_elapsed(fld, 'Before')

In [52]:
fld = 'Promo'
df = df.sort_values(['Store', 'Date'])
get_elapsed(fld, 'After')
df = df.sort_values(['Store', 'Date'], ascending=[True, False])
get_elapsed(fld, 'Before')

We're going to set the active index to Date.

In [53]:
df

,Date,Store,Promo,StateHoliday,SchoolHoliday,AfterSchoolHoliday,BeforeSchoolHoliday,AfterStateHoliday,BeforeStateHoliday,AfterPromo,BeforePromo
0,2015-09-17,1,1,False,0,13.0,NaN,105.0,NaN,0.0,0.0
856,2015-09-16,1,1,False,0,12.0,NaN,104.0,NaN,0.0,0.0
1712,2015-09-15,1,1,False,0,11.0,NaN,103.0,NaN,0.0,0.0
2568,2015-09-14,1,1,False,0,10.0,NaN,102.0,NaN,0.0,0.0
3424,2015-09-13,1,0,False,0,9.0,NaN,101.0,NaN,9.0,-1.0
4280,2015-09-12,1,0,False,0,8.0,NaN,100.0,NaN,8.0,-2.0
5136,2015-09-11,1,0,False,0,7.0,NaN,99.0,NaN,7.0,-3.0
5992,2015-09-10,1,0,False,0,6.0,NaN,98.0,NaN,6.0,-4.0
6848,2015-09-09,1,0,False,0,5.0,NaN,97.0,NaN,5.0,-5.0
7704,2015-09-08,1,0,False,0,4.0,NaN,96.0,NaN,4.0,-6.0


We're going to set the active index to Date.

In [54]:
df = df.set_index("Date")

In [55]:
df

,Store,Promo,StateHoliday,SchoolHoliday,AfterSchoolHoliday,BeforeSchoolHoliday,AfterStateHoliday,BeforeStateHoliday,AfterPromo,BeforePromo
Date,,,,,,,,,,
2015-09-17,1,1,False,0,13.0,NaN,105.0,NaN,0.0,0.0
2015-09-16,1,1,False,0,12.0,NaN,104.0,NaN,0.0,0.0
2015-09-15,1,1,False,0,11.0,NaN,103.0,NaN,0.0,0.0
2015-09-14,1,1,False,0,10.0,NaN,102.0,NaN,0.0,0.0
2015-09-13,1,0,False,0,9.0,NaN,101.0,NaN,9.0,-1.0
2015-09-12,1,0,False,0,8.0,NaN,100.0,NaN,8.0,-2.0
2015-09-11,1,0,False,0,7.0,NaN,99.0,NaN,7.0,-3.0
2015-09-10,1,0,False,0,6.0,NaN,98.0,NaN,6.0,-4.0
2015-09-09,1,0,False,0,5.0,NaN,97.0,NaN,5.0,-5.0


Then set null values from elapsed field calculations to 0.

In [56]:
columns = ['SchoolHoliday', 'StateHoliday', 'Promo']

In [57]:
for o in ['Before', 'After']:
    for p in columns:
        a = o+p
        df[a] = df[a].fillna(0).astype(int)

Next we'll demonstrate window functions in pandas to calculate rolling quantities.

Here we're sorting by date (sort_index()) and counting the number of events of interest (sum()) defined in columns in the following week (rolling()), grouped by Store (groupby()). We do the same in the opposite direction.

In [58]:
bwd = df[['Store']+columns].sort_index().groupby("Store").rolling(7, min_periods=1).sum()

In [59]:
bwd

Store  SchoolHoliday  StateHoliday  Promo
Store Date                                                  
1     2013-01-01     1.0            1.0           1.0    0.0
      2013-01-02     2.0            2.0           1.0    0.0
      2013-01-03     3.0            3.0           1.0    0.0
      2013-01-04     4.0            4.0           1.0    0.0
      2013-01-05     5.0            5.0           1.0    0.0
      2013-01-06     6.0            6.0           1.0    0.0
      2013-01-07     7.0            7.0           1.0    1.0
      2013-01-08     7.0            7.0           0.0    2.0
      2013-01-09     7.0            7.0           0.0    3.0
      2013-01-10     7.0            7.0           0.0    4.0
      2013-01-11     7.0            7.0           0.0    5.0
      2013-01-12     7.0            6.0           0.0    5.0
      2013-01-13     7.0            5.0           0.0    5.0
      2013-01-14     7.0            4.0           0.0    4.0
      2013-01-15     7.0            3.0           0.0    3.0
      2013-01-16     7.0            2.0           0.0    2.0
      2013-01-17     7.0            1.0           0.0    1.0
      2013-01-18     7.0            0.0           0.0    0.0
      2013-01-19     7.0            0.0           0.0    0.0
      2013-01-20     7.0            0.0           0.0    0.0
      2013-01-21     7.0            0.0           0.0    1.0
      2013-01-22     7.0            0.0           0.0    2.0
      2013-01-23     7.0            0.0           0.0    3.0
      2013-01-24     7.0            0.0           0.0    4.0
      2013-01-25     7.0            0.0           0.0    5.0
      2013-01-26     7.0            0.0           0.0    5.0
      2013-01-27     7.0            0.0           0.0    5.0
      2013-01-28     7.0            0.0           0.0    4.0
      2013-01-29     7.0            0.0           0.0    3.0
      2013-01-30     7.0            0.0           0.0    2.0
...                  ...            ...           ...    ...
1115  2015-08-19  7805.0            7.0           0.0    3.0
      2015-08-20  7805.0            7.0           0.0    4.0
      2015-08-21  7805.0            7.0           0.0    5.0
      2015-08-22  7805.0            7.0           0.0    5.0
      2015-08-23  7805.0            7.0           0.0    5.0
      2015-08-24  7805.0            7.0           0.0    4.0
      2015-08-25  7805.0            7.0           0.0    3.0
      2015-08-26  7805.0            7.0           0.0    2.0
      2015-08-27  7805.0            7.0           0.0    1.0
      2015-08-28  7805.0            7.0           0.0    0.0
      2015-08-29  7805.0            7.0           0.0    0.0
      2015-08-30  7805.0            7.0           0.0    0.0
      2015-08-31  7805.0            7.0           0.0    1.0
      2015-09-01  7805.0            7.0           0.0    2.0
      2015-09-02  7805.0            7.0           0.0    3.0
      2015-09-03  7805.0            7.0           0.0    4.0
      2015-09-04  7805.0            7.0           0.0    5.0
      2015-09-05  7805.0            6.0           0.0    5.0
      2015-09-06  7805.0            5.0           0.0    5.0
      2015-09-07  7805.0            4.0           0.0    4.0
      2015-09-08  7805.0            3.0           0.0    3.0
      2015-09-09  7805.0            2.0           0.0    2.0
      2015-09-10  7805.0            1.0           0.0    1.0
      2015-09-11  7805.0            0.0           0.0    0.0
      2015-09-12  7805.0            0.0           0.0    0.0
      2015-09-13  7805.0            0.0           0.0    0.0
      2015-09-14  7805.0            0.0           0.0    1.0
      2015-09-15  7805.0            0.0           0.0    2.0
      2015-09-16  7805.0            0.0           0.0    3.0
      2015-09-17  7805.0            0.0           0.0    4.0

[1058297 rows x 4 columns]

In [60]:
fwd = df[['Store']+columns].sort_index(ascending=False
                                      ).groupby("Store").rolling(7, min_periods=1).sum()

In [61]:
fwd

Store  SchoolHoliday  StateHoliday  Promo
Store Date                                                  
1     2015-09-17     1.0            0.0           0.0    1.0
      2015-09-16     2.0            0.0           0.0    2.0
      2015-09-15     3.0            0.0           0.0    3.0
      2015-09-14     4.0            0.0           0.0    4.0
      2015-09-13     5.0            0.0           0.0    4.0
      2015-09-12     6.0            0.0           0.0    4.0
      2015-09-11     7.0            0.0           0.0    4.0
      2015-09-10     7.0            0.0           0.0    3.0
      2015-09-09     7.0            0.0           0.0    2.0
      2015-09-08     7.0            0.0           0.0    1.0
      2015-09-07     7.0            0.0           0.0    0.0
      2015-09-06     7.0            0.0           0.0    0.0
      2015-09-05     7.0            0.0           0.0    0.0
      2015-09-04     7.0            1.0           0.0    1.0
      2015-09-03     7.0            2.0           0.0    2.0
      2015-09-02     7.0            3.0           0.0    3.0
      2015-09-01     7.0            4.0           0.0    4.0
      2015-08-31     7.0            5.0           0.0    5.0
      2015-08-30     7.0            6.0           0.0    5.0
      2015-08-29     7.0            7.0           0.0    5.0
      2015-08-28     7.0            7.0           0.0    4.0
      2015-08-27     7.0            7.0           0.0    3.0
      2015-08-26     7.0            7.0           0.0    2.0
      2015-08-25     7.0            7.0           0.0    1.0
      2015-08-24     7.0            7.0           0.0    0.0
      2015-08-23     7.0            7.0           0.0    0.0
      2015-08-22     7.0            7.0           0.0    0.0
      2015-08-21     7.0            7.0           0.0    1.0
      2015-08-20     7.0            7.0           0.0    2.0
      2015-08-19     7.0            7.0           0.0    3.0
...                  ...            ...           ...    ...
1115  2013-01-30  7805.0            0.0           0.0    2.0
      2013-01-29  7805.0            0.0           0.0    1.0
      2013-01-28  7805.0            0.0           0.0    0.0
      2013-01-27  7805.0            0.0           0.0    0.0
      2013-01-26  7805.0            0.0           0.0    0.0
      2013-01-25  7805.0            0.0           0.0    1.0
      2013-01-24  7805.0            0.0           0.0    2.0
      2013-01-23  7805.0            0.0           0.0    3.0
      2013-01-22  7805.0            0.0           0.0    4.0
      2013-01-21  7805.0            0.0           0.0    5.0
      2013-01-20  7805.0            0.0           0.0    5.0
      2013-01-19  7805.0            0.0           0.0    5.0
      2013-01-18  7805.0            0.0           0.0    4.0
      2013-01-17  7805.0            0.0           0.0    3.0
      2013-01-16  7805.0            0.0           0.0    2.0
      2013-01-15  7805.0            0.0           0.0    1.0
      2013-01-14  7805.0            0.0           0.0    0.0
      2013-01-13  7805.0            0.0           0.0    0.0
      2013-01-12  7805.0            0.0           0.0    0.0
      2013-01-11  7805.0            1.0           0.0    1.0
      2013-01-10  7805.0            2.0           0.0    2.0
      2013-01-09  7805.0            3.0           0.0    3.0
      2013-01-08  7805.0            4.0           0.0    4.0
      2013-01-07  7805.0            5.0           0.0    5.0
      2013-01-06  7805.0            6.0           0.0    5.0
      2013-01-05  7805.0            7.0           0.0    5.0
      2013-01-04  7805.0            7.0           0.0    4.0
      2013-01-03  7805.0            7.0           0.0    3.0
      2013-01-02  7805.0            7.0           0.0    2.0
      2013-01-01  7805.0            7.0           1.0    1.0

[1058297 rows x 4 columns]

Next we want to drop the Store indices grouped together in the window function.

Often in pandas, there is an option to do this in place. This is time and memory efficient when working with large datasets.

In [62]:
bwd

Store  SchoolHoliday  StateHoliday  Promo
Store Date                                                  
1     2013-01-01     1.0            1.0           1.0    0.0
      2013-01-02     2.0            2.0           1.0    0.0
      2013-01-03     3.0            3.0           1.0    0.0
      2013-01-04     4.0            4.0           1.0    0.0
      2013-01-05     5.0            5.0           1.0    0.0
      2013-01-06     6.0            6.0           1.0    0.0
      2013-01-07     7.0            7.0           1.0    1.0
      2013-01-08     7.0            7.0           0.0    2.0
      2013-01-09     7.0            7.0           0.0    3.0
      2013-01-10     7.0            7.0           0.0    4.0
      2013-01-11     7.0            7.0           0.0    5.0
      2013-01-12     7.0            6.0           0.0    5.0
      2013-01-13     7.0            5.0           0.0    5.0
      2013-01-14     7.0            4.0           0.0    4.0
      2013-01-15     7.0            3.0           0.0    3.0
      2013-01-16     7.0            2.0           0.0    2.0
      2013-01-17     7.0            1.0           0.0    1.0
      2013-01-18     7.0            0.0           0.0    0.0
      2013-01-19     7.0            0.0           0.0    0.0
      2013-01-20     7.0            0.0           0.0    0.0
      2013-01-21     7.0            0.0           0.0    1.0
      2013-01-22     7.0            0.0           0.0    2.0
      2013-01-23     7.0            0.0           0.0    3.0
      2013-01-24     7.0            0.0           0.0    4.0
      2013-01-25     7.0            0.0           0.0    5.0
      2013-01-26     7.0            0.0           0.0    5.0
      2013-01-27     7.0            0.0           0.0    5.0
      2013-01-28     7.0            0.0           0.0    4.0
      2013-01-29     7.0            0.0           0.0    3.0
      2013-01-30     7.0            0.0           0.0    2.0
...                  ...            ...           ...    ...
1115  2015-08-19  7805.0            7.0           0.0    3.0
      2015-08-20  7805.0            7.0           0.0    4.0
      2015-08-21  7805.0            7.0           0.0    5.0
      2015-08-22  7805.0            7.0           0.0    5.0
      2015-08-23  7805.0            7.0           0.0    5.0
      2015-08-24  7805.0            7.0           0.0    4.0
      2015-08-25  7805.0            7.0           0.0    3.0
      2015-08-26  7805.0            7.0           0.0    2.0
      2015-08-27  7805.0            7.0           0.0    1.0
      2015-08-28  7805.0            7.0           0.0    0.0
      2015-08-29  7805.0            7.0           0.0    0.0
      2015-08-30  7805.0            7.0           0.0    0.0
      2015-08-31  7805.0            7.0           0.0    1.0
      2015-09-01  7805.0            7.0           0.0    2.0
      2015-09-02  7805.0            7.0           0.0    3.0
      2015-09-03  7805.0            7.0           0.0    4.0
      2015-09-04  7805.0            7.0           0.0    5.0
      2015-09-05  7805.0            6.0           0.0    5.0
      2015-09-06  7805.0            5.0           0.0    5.0
      2015-09-07  7805.0            4.0           0.0    4.0
      2015-09-08  7805.0            3.0           0.0    3.0
      2015-09-09  7805.0            2.0           0.0    2.0
      2015-09-10  7805.0            1.0           0.0    1.0
      2015-09-11  7805.0            0.0           0.0    0.0
      2015-09-12  7805.0            0.0           0.0    0.0
      2015-09-13  7805.0            0.0           0.0    0.0
      2015-09-14  7805.0            0.0           0.0    1.0
      2015-09-15  7805.0            0.0           0.0    2.0
      2015-09-16  7805.0            0.0           0.0    3.0
      2015-09-17  7805.0            0.0           0.0    4.0

[1058297 rows x 4 columns]

In [63]:
bwd.drop('Store',1,inplace=True)
bwd.reset_index(inplace=True)

In [64]:
bwd

,Store,Date,SchoolHoliday,StateHoliday,Promo
0,1,2013-01-01,1.0,1.0,0.0
1,1,2013-01-02,2.0,1.0,0.0
2,1,2013-01-03,3.0,1.0,0.0
3,1,2013-01-04,4.0,1.0,0.0
4,1,2013-01-05,5.0,1.0,0.0
5,1,2013-01-06,6.0,1.0,0.0
6,1,2013-01-07,7.0,1.0,1.0
7,1,2013-01-08,7.0,0.0,2.0
8,1,2013-01-09,7.0,0.0,3.0
9,1,2013-01-10,7.0,0.0,4.0


In [65]:
fwd

Store  SchoolHoliday  StateHoliday  Promo
Store Date                                                  
1     2015-09-17     1.0            0.0           0.0    1.0
      2015-09-16     2.0            0.0           0.0    2.0
      2015-09-15     3.0            0.0           0.0    3.0
      2015-09-14     4.0            0.0           0.0    4.0
      2015-09-13     5.0            0.0           0.0    4.0
      2015-09-12     6.0            0.0           0.0    4.0
      2015-09-11     7.0            0.0           0.0    4.0
      2015-09-10     7.0            0.0           0.0    3.0
      2015-09-09     7.0            0.0           0.0    2.0
      2015-09-08     7.0            0.0           0.0    1.0
      2015-09-07     7.0            0.0           0.0    0.0
      2015-09-06     7.0            0.0           0.0    0.0
      2015-09-05     7.0            0.0           0.0    0.0
      2015-09-04     7.0            1.0           0.0    1.0
      2015-09-03     7.0            2.0           0.0    2.0
      2015-09-02     7.0            3.0           0.0    3.0
      2015-09-01     7.0            4.0           0.0    4.0
      2015-08-31     7.0            5.0           0.0    5.0
      2015-08-30     7.0            6.0           0.0    5.0
      2015-08-29     7.0            7.0           0.0    5.0
      2015-08-28     7.0            7.0           0.0    4.0
      2015-08-27     7.0            7.0           0.0    3.0
      2015-08-26     7.0            7.0           0.0    2.0
      2015-08-25     7.0            7.0           0.0    1.0
      2015-08-24     7.0            7.0           0.0    0.0
      2015-08-23     7.0            7.0           0.0    0.0
      2015-08-22     7.0            7.0           0.0    0.0
      2015-08-21     7.0            7.0           0.0    1.0
      2015-08-20     7.0            7.0           0.0    2.0
      2015-08-19     7.0            7.0           0.0    3.0
...                  ...            ...           ...    ...
1115  2013-01-30  7805.0            0.0           0.0    2.0
      2013-01-29  7805.0            0.0           0.0    1.0
      2013-01-28  7805.0            0.0           0.0    0.0
      2013-01-27  7805.0            0.0           0.0    0.0
      2013-01-26  7805.0            0.0           0.0    0.0
      2013-01-25  7805.0            0.0           0.0    1.0
      2013-01-24  7805.0            0.0           0.0    2.0
      2013-01-23  7805.0            0.0           0.0    3.0
      2013-01-22  7805.0            0.0           0.0    4.0
      2013-01-21  7805.0            0.0           0.0    5.0
      2013-01-20  7805.0            0.0           0.0    5.0
      2013-01-19  7805.0            0.0           0.0    5.0
      2013-01-18  7805.0            0.0           0.0    4.0
      2013-01-17  7805.0            0.0           0.0    3.0
      2013-01-16  7805.0            0.0           0.0    2.0
      2013-01-15  7805.0            0.0           0.0    1.0
      2013-01-14  7805.0            0.0           0.0    0.0
      2013-01-13  7805.0            0.0           0.0    0.0
      2013-01-12  7805.0            0.0           0.0    0.0
      2013-01-11  7805.0            1.0           0.0    1.0
      2013-01-10  7805.0            2.0           0.0    2.0
      2013-01-09  7805.0            3.0           0.0    3.0
      2013-01-08  7805.0            4.0           0.0    4.0
      2013-01-07  7805.0            5.0           0.0    5.0
      2013-01-06  7805.0            6.0           0.0    5.0
      2013-01-05  7805.0            7.0           0.0    5.0
      2013-01-04  7805.0            7.0           0.0    4.0
      2013-01-03  7805.0            7.0           0.0    3.0
      2013-01-02  7805.0            7.0           0.0    2.0
      2013-01-01  7805.0            7.0           1.0    1.0

[1058297 rows x 4 columns]

In [66]:
fwd.drop('Store',1,inplace=True)
fwd.reset_index(inplace=True)

In [67]:
fwd

,Store,Date,SchoolHoliday,StateHoliday,Promo
0,1,2015-09-17,0.0,0.0,1.0
1,1,2015-09-16,0.0,0.0,2.0
2,1,2015-09-15,0.0,0.0,3.0
3,1,2015-09-14,0.0,0.0,4.0
4,1,2015-09-13,0.0,0.0,4.0
5,1,2015-09-12,0.0,0.0,4.0
6,1,2015-09-11,0.0,0.0,4.0
7,1,2015-09-10,0.0,0.0,3.0
8,1,2015-09-09,0.0,0.0,2.0
9,1,2015-09-08,0.0,0.0,1.0


In [68]:
df

,Store,Promo,StateHoliday,SchoolHoliday,AfterSchoolHoliday,BeforeSchoolHoliday,AfterStateHoliday,BeforeStateHoliday,AfterPromo,BeforePromo
Date,,,,,,,,,,
2015-09-17,1,1,False,0,13,0,105,0,0,0
2015-09-16,1,1,False,0,12,0,104,0,0,0
2015-09-15,1,1,False,0,11,0,103,0,0,0
2015-09-14,1,1,False,0,10,0,102,0,0,0
2015-09-13,1,0,False,0,9,0,101,0,9,-1
2015-09-12,1,0,False,0,8,0,100,0,8,-2
2015-09-11,1,0,False,0,7,0,99,0,7,-3
2015-09-10,1,0,False,0,6,0,98,0,6,-4
2015-09-09,1,0,False,0,5,0,97,0,5,-5


In [69]:
df.reset_index(inplace=True)

In [70]:
df

,Date,Store,Promo,StateHoliday,SchoolHoliday,AfterSchoolHoliday,BeforeSchoolHoliday,AfterStateHoliday,BeforeStateHoliday,AfterPromo,BeforePromo
0,2015-09-17,1,1,False,0,13,0,105,0,0,0
1,2015-09-16,1,1,False,0,12,0,104,0,0,0
2,2015-09-15,1,1,False,0,11,0,103,0,0,0
3,2015-09-14,1,1,False,0,10,0,102,0,0,0
4,2015-09-13,1,0,False,0,9,0,101,0,9,-1
5,2015-09-12,1,0,False,0,8,0,100,0,8,-2
6,2015-09-11,1,0,False,0,7,0,99,0,7,-3
7,2015-09-10,1,0,False,0,6,0,98,0,6,-4
8,2015-09-09,1,0,False,0,5,0,97,0,5,-5
9,2015-09-08,1,0,False,0,4,0,96,0,4,-6


Now we'll merge these values onto the df.

In [71]:
df = df.merge(bwd, 'left', ['Date', 'Store'], suffixes=['', '_bw'])
df = df.merge(fwd, 'left', ['Date', 'Store'], suffixes=['', '_fw'])

In [72]:
df

,Date,Store,Promo,StateHoliday,SchoolHoliday,AfterSchoolHoliday,BeforeSchoolHoliday,AfterStateHoliday,BeforeStateHoliday,AfterPromo,BeforePromo,SchoolHoliday_bw,StateHoliday_bw,Promo_bw,SchoolHoliday_fw,StateHoliday_fw,Promo_fw
0,2015-09-17,1,1,False,0,13,0,105,0,0,0,0.0,0.0,4.0,0.0,0.0,1.0
1,2015-09-16,1,1,False,0,12,0,104,0,0,0,0.0,0.0,3.0,0.0,0.0,2.0
2,2015-09-15,1,1,False,0,11,0,103,0,0,0,0.0,0.0,2.0,0.0,0.0,3.0
3,2015-09-14,1,1,False,0,10,0,102,0,0,0,0.0,0.0,1.0,0.0,0.0,4.0
4,2015-09-13,1,0,False,0,9,0,101,0,9,-1,0.0,0.0,0.0,0.0,0.0,4.0
5,2015-09-12,1,0,False,0,8,0,100,0,8,-2,0.0,0.0,0.0,0.0,0.0,4.0
6,2015-09-11,1,0,False,0,7,0,99,0,7,-3,0.0,0.0,0.0,0.0,0.0,4.0
7,2015-09-10,1,0,False,0,6,0,98,0,6,-4,1.0,0.0,1.0,0.0,0.0,3.0
8,2015-09-09,1,0,False,0,5,0,97,0,5,-5,2.0,0.0,2.0,0.0,0.0,2.0
9,2015-09-08,1,0,False,0,4,0,96,0,4,-6,3.0,0.0,3.0,0.0,0.0,1.0


In [73]:
df.drop(columns,1,inplace=True)

In [74]:
df.head()

,Date,Store,AfterSchoolHoliday,BeforeSchoolHoliday,AfterStateHoliday,BeforeStateHoliday,AfterPromo,BeforePromo,SchoolHoliday_bw,StateHoliday_bw,Promo_bw,SchoolHoliday_fw,StateHoliday_fw,Promo_fw
0,2015-09-17,1,13,0,105,0,0,0,0.0,0.0,4.0,0.0,0.0,1.0
1,2015-09-16,1,12,0,104,0,0,0,0.0,0.0,3.0,0.0,0.0,2.0
2,2015-09-15,1,11,0,103,0,0,0,0.0,0.0,2.0,0.0,0.0,3.0
3,2015-09-14,1,10,0,102,0,0,0,0.0,0.0,1.0,0.0,0.0,4.0
4,2015-09-13,1,9,0,101,0,9,-1,0.0,0.0,0.0,0.0,0.0,4.0
